In [1]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import lxml.html as lh
import re
import os
import pandas as pd
import numpy as np
from distutils.version import LooseVersion
from distutils.version import StrictVersion


In [2]:
####change the language name


In [4]:
#crawl the vocaholic web page
def vocaholic_crawler(lan):
    url= "http://aac-srvtts-tools.nuance.com/vocaholic/fe/release/?language=" + lan
    #url=urlv+lan
    browser = webdriver.Chrome()

    browser.get(url)
    time.sleep(3)
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")

    #print(len(soup.find_all("table")))
    table=soup.find("table", {"id": "myTable"})
    #print(table)

    browser.close()
    browser.quit()

    #get the table
    table_rows = table.find_all('tr')
    rows=[]
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        rows.append(row)

    cols=soup.find_all("th")
    col_names=[]
    for th in cols:
        col=th.text
        col_names.append(col)
    #print(col_names[:16])
    #print(len(col_names[:16]))
    cols = col_names[:17]
    df=pd.DataFrame(rows[2:],columns=cols)
    voices=list(set(df['Voice'].tolist()))
    return voices, df

    #cols=info_tr.findall('th')
    #print(cols)

In [55]:
#get the list of vops with the latst FE 
def loop_voices(v):
    fe_dict={}
    i = 0
    n = len(v)
    while i <n:
        df_v=df[df['Voice']==v[i]].replace('NA', np.nan)
        #print(df_v.head())
        fe_v= df_v['CLC Version'].max()
        fe_dict[v[i]]=fe_v
        i +=1
        #print(fe_v)

    #return fe_dict
    vops=[]
    for key, value in fe_dict.items():
        pub_date=df[df['Voice']==key] [ df['CLC Version']==value][ 'Published Date'].max()
        used=df[df['Voice']==key][df['Published Date']==pub_date]['Used In'].tolist()
        uesd_in=used[0].split('\n')
        ehs = [v for v in uesd_in if ('_embedded-high-' in v) & ('-SNAPSHOT' not in v) ]
        if len(ehs) == 0:
            vops.append([v for v in uesd_in][1] )
        else:
            vops.append(ehs[-1])
    #v=open(ver_path,'w+')
    #v.writelines()
    return vops
        


[u' frc_nicolas_embedded-high-2.0.0', u' frc_chantal_embedded-high-1.0.7', u' frc_amelie_embedded-high-1.0.5', u' frc_amelie-ml_embedded-high-1.0.3']


c:\_tools\python27\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
c:\_tools\python27\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [50]:
#convert the vop names into versions.txt
def vop_to_version(vop):
    vop=str(vop.strip(' '))
    num=vop.split('_')
    v='vop#'+ num[0]+'#'+num[1]+'#22#'+num[2]
    v=v.replace('-','#')
    if 'embedded#' in v:
        v=v.replace('embedded#','embedded-')
    if '#ml'in v:
        v=v.replace('#ml','-ml')
    return v
    
#print(vop_to_version(vops_latest[2]))


['vop#spe#monica#22#embedded-high#1.2.3\n', 'vop#spe#djingo#22#embedded-high#1.2.0\n', 'vop#spe#monica-ml#22#embedded-high#2.1.3\n', 'vop#spe#marisol-ml#22#embedded-high#3.0.0\n', 'vop#spe#jorge#22#embedded-high#2.1.0\n', 'vop#spe#marisol#22#embedded-high#2.0.0\n']


In [5]:
if __name__ == '__main__':
    lan="frc"
    v, df = vocaholic_crawler(lan)
    df.to_csv('FE_check.csv',sep=';' ,index=False,encoding='utf-8_sig')
    
    vops_latest=loop_voices(v)
    print(vops_latest)
    
    #save to the versions.txt
    ver_path='C:\_workspace\\ALD\\multiple_stage\\test\\versions.txt'    
    version_lines=[vop_to_version(vops)+'\n' for vops in vops_latest]
    print(version_lines)
    
    versions=open(ver_path,'w+')
    versions.writelines(version_lines)
    versions.close()
    
    
    #run stage.ext to the ref stages
    cmd='stage '+ ver_path +' . --frontend_only yes'
    os.system(cmd)


[u'nicolas', u'chantal', u'amelie', u'amelie-ml']


In [65]:
def run_gendata(user_path, cfg):

-1

###########################
INFO     :: # Command line:
 E:\LBTools\2.1.0\bin\gendata.exe -t sharedclc -l spe -v marisol-ml -c cfg3 -o E:\Users\Shanshan.Xu\builds\\SPE\Marisol-ml\cfg3 -s E:\Users\Shanshan.Xu\hg\clc --ignore-errors
 
############################

to do:
    1. set the batch
    2. get the target stages
    3. deleted unwanted files.

In [19]:
voice = 'amelie-ml'
user_path=r'E:\Users\Shanshan.Xu'
clc_path = os.path.join(user_path,'hg','clc')
cfg='cfg3'
output_path= os.path.join(user_path, 'builds', lan,voice,cfg)
print output_path
cmd = r'E:\LBTools\2.1.0\bin\gendata.exe -t sharedclc -l ' +  lan + ' -v ' + voice + r' -c ' + cfg + ' -o ' + output_path + ' -s ' + clc_path + ' --ignore-errors'
print cmd

E:\Users\Shanshan.Xu\builds\frc\amelie-ml\cfg3
E:\LBTools\2.1.0\bin\gendata.exe -t sharedclc -l frc -v amelie-ml -c cfg3 -o E:\Users\Shanshan.Xu\builds\frc\amelie-ml\cfg3 -s E:\Users\Shanshan.Xu\hg\clc --ignore-errors


In [ ]:
l=os.listdir()